### 지역별 전기차 증가율에 따른 지역선정

In [1]:
import pandas as pd
import folium
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import font_manager, rc
import seaborn as sns
import platform

In [2]:
# seaborn 설정 리셋
sns.reset_defaults()

# 폰트설정
if platform.system() == 'Windows' :
    path = 'c:/Windows/Fonts/malgun.ttf'
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
elif platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
else :
    print('Check your OS System')
    
# 그래프에 마이너스 표시
matplotlib.rcParams['axes.unicode_minus'] = False

In [6]:
charge = pd.read_csv('./data/rawdata/전국_충전소_현황(기존데이터).csv')
charge

,운영기관,충전소,충전기ID,충전기타입,지역,시군구,주소,이용가능시간,이용자 제한,급속충전량,비고
0,환경부(한국자동차환경협회),신북보건지소공영주차장,21,DC콤보,전라남도,영암군,전라남도 영암군 신북면 간은정로 21-7,24시간 이용가능,NaN,급속(200kW동시),NaN
1,환경부(한국자동차환경협회),신북보건지소공영주차장,22,DC콤보,전라남도,영암군,전라남도 영암군 신북면 간은정로 21-7,24시간 이용가능,NaN,급속(200kW동시),NaN
2,환경부(한국자동차환경협회),한국생산기술연구원 대경본부바이오메디칼생산기술센터,1,DC콤보,경상북도,영천시,경상북도 영천시 양호길 59,24시간 이용가능,NaN,급속(100kW단독),NaN
3,환경부(한국자동차환경협회),한국생산기술연구원 대경본부바이오메디칼생산기술센터,2,DC콤보,경상북도,영천시,경상북도 영천시 양호길 59,24시간 이용가능,NaN,급속(100kW단독),NaN
4,환경부(한국자동차환경협회),(구)거진수산센터 주차장,11,DC콤보,강원도,고성군,강원도 고성군 거진읍 거진리 22-93,24시간 이용가능,NaN,급속(200kW동시),NaN
...,...,...,...,...,...,...,...,...,...,...,...
121020,한화솔루션,한화리조트 평창,5,AC완속,강원도,평창군,강원도 평창군 봉평면 태기로 228-33한화리조트 평창(946-7 한화리조트),24시간 이용가능,NaN,NaN,NaN
121021,한화솔루션,한화리조트 평창,6,AC완속,강원도,평창군,강원도 평창군 봉평면 태기로 228-33한화리조트 평창(946-7 한화리조트),24시간 이용가능,NaN,NaN,NaN
121022,한화솔루션,한화리조트 평창,7,AC완속,강원도,평창군,강원도 평창군 봉평면 태기로 228-33한화리조트 평창(946-7 한화리조트),24시간 이용가능,NaN,NaN,NaN
121023,한화솔루션,한화리조트 평창,8,AC완속,강원도,평창군,강원도 평창군 봉평면 태기로 228-33한화리조트 평창(946-7 한화리조트),24시간 이용가능,NaN,NaN,NaN


In [7]:
# 지역 컬럼 이상 문자열 처리 
charge['지역'] = charge['지역'].str.strip()

In [8]:
# 필요지역(경기도) 및 필요 컬럼 추출
charge_gyeonggi = charge[charge['지역'] == '경기도'].drop(columns=['급속충전량','비고','이용가능시간']).reset_index(drop=True)

In [9]:
# 결측치, 이상치 확인
charge_gyeonggi.info()
charge_gyeonggi.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30116 entries, 0 to 30115
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   운영기관    30116 non-null  object
 1   충전소     30116 non-null  object
 2   충전기ID   30116 non-null  int64 
 3   충전기타입   30116 non-null  object
 4   지역      30116 non-null  object
 5   시군구     30105 non-null  object
 6   주소      30116 non-null  object
 7   이용자 제한  18772 non-null  object
dtypes: int64(1), object(7)
memory usage: 1.8+ MB


,충전기ID
count,30116.000000
mean,5.017399
std,7.361474
min,0.000000
25%,1.000000
50%,3.000000
75%,5.000000
max,98.000000


In [10]:
# 중복치 확인
charge_gyeonggi[charge_gyeonggi.duplicated()]

,운영기관,충전소,충전기ID,충전기타입,지역,시군구,주소,이용자 제한
20153,파워큐브,별내효성해링턴코트,1,AC완속,경기도,남양주시,경기도 남양주시 별내3로 185(효성해링턴코트 아파트),거주자외 출입제한
20155,파워큐브,별내효성해링턴코트,2,AC완속,경기도,남양주시,경기도 남양주시 별내3로 185(효성해링턴코트 아파트),거주자외 출입제한
20157,파워큐브,별내효성해링턴코트,3,AC완속,경기도,남양주시,경기도 남양주시 별내3로 185(효성해링턴코트 아파트),거주자외 출입제한
20159,파워큐브,별내효성해링턴코트,4,AC완속,경기도,남양주시,경기도 남양주시 별내3로 185(효성해링턴코트 아파트),거주자외 출입제한
20161,파워큐브,별내효성해링턴코트,5,AC완속,경기도,남양주시,경기도 남양주시 별내3로 185(효성해링턴코트 아파트),거주자외 출입제한
20163,파워큐브,별내효성해링턴코트,6,AC완속,경기도,남양주시,경기도 남양주시 별내3로 185(효성해링턴코트 아파트),거주자외 출입제한
20224,파워큐브,북한강코아루입주자대표회의,1,AC완속,경기도,가평군,경기도 가평군 설악면 자잠로 86(663 북한강코아루),거주자외 출입제한
20226,파워큐브,북한강코아루입주자대표회의,2,AC완속,경기도,가평군,경기도 가평군 설악면 자잠로 86(663 북한강코아루),거주자외 출입제한
20528,파워큐브,송탄역서희스타힐스아파트,1,AC완속,경기도,평택시,경기도 평택시 목천로 65-15(송탄역 서희스타힐스),거주자외 출입제한
20530,파워큐브,송탄역서희스타힐스아파트,2,AC완속,경기도,평택시,경기도 평택시 목천로 65-15(송탄역 서희스타힐스),거주자외 출입제한


In [14]:
# 충전기 ID값으로 보아 실제 중복치로 판단하여 중복치 제거
charge_gyeonggi.drop_duplicates(inplace=True)
charge_gyeonggi[charge_gyeonggi.duplicated()]

,운영기관,충전소,충전기ID,충전기타입,지역,시군구,주소,이용자 제한


In [15]:
# 이용자 제한의 결측치를 24시간 이용 가능으로 판단 하여 '제한없음'으로 값 재 정의
charge_gyeonggi['이용자 제한'].fillna('제한없음',inplace=True)

In [16]:
# 시군구 컬럼의 결측치 확인
charge_gyeonggi[charge_gyeonggi['시군구'].isna()]

,운영기관,충전소,충전기ID,충전기타입,지역,시군구,주소,이용자 제한
877,제주전기자동차서비스,CU동탄금곡로점,1,DC콤보,경기도,NaN,경기도 화성시,제한없음
891,제주전기자동차서비스,N2WASH 일산전기차충전소,1,DC차데모+DC콤보,경기도,NaN,경기도 고양시,제한없음
892,제주전기자동차서비스,N2WASH 일산전기차충전소,2,DC차데모+DC콤보,경기도,NaN,경기도 고양시,제한없음
893,제주전기자동차서비스,N2WASH 일산전기차충전소,3,AC완속,경기도,NaN,경기도 고양시,제한없음
894,제주전기자동차서비스,N2WASH 일산전기차충전소,4,AC완속,경기도,NaN,경기도 고양시,제한없음
3553,한국전기차충전서비스,금곡제1공영주차장,1,DC차데모+DC콤보,경기도,NaN,경기도 남양주시,제한없음
3647,한국전기차충전서비스,다산제2공영주차장,1,DC차데모+DC콤보,경기도,NaN,경기도 남양주시,제한없음
3648,한국전기차충전서비스,다산제2공영주차장,2,AC완속,경기도,NaN,경기도 남양주시,제한없음
3771,한국전기차충전서비스,문원동 공영주차장,1,DC차데모+DC콤보,경기도,NaN,경기도 과천시,제한없음
3818,한국전기차충전서비스,별내제1공영주차장,1,DC차데모+DC콤보,경기도,NaN,경기도 남양주시,제한없음


In [17]:
# 정확한 결측치 확인을 위해 문자열 슬라이싱을 통한 재확인
charge_gyeonggi['주소2'] = charge_gyeonggi['주소'].str[9:]
charge_gyeonggi[charge_gyeonggi['주소2'] == '']

,운영기관,충전소,충전기ID,충전기타입,지역,시군구,주소,이용자 제한,주소2
877,제주전기자동차서비스,CU동탄금곡로점,1,DC콤보,경기도,NaN,경기도 화성시,제한없음,
891,제주전기자동차서비스,N2WASH 일산전기차충전소,1,DC차데모+DC콤보,경기도,NaN,경기도 고양시,제한없음,
892,제주전기자동차서비스,N2WASH 일산전기차충전소,2,DC차데모+DC콤보,경기도,NaN,경기도 고양시,제한없음,
893,제주전기자동차서비스,N2WASH 일산전기차충전소,3,AC완속,경기도,NaN,경기도 고양시,제한없음,
894,제주전기자동차서비스,N2WASH 일산전기차충전소,4,AC완속,경기도,NaN,경기도 고양시,제한없음,
3553,한국전기차충전서비스,금곡제1공영주차장,1,DC차데모+DC콤보,경기도,NaN,경기도 남양주시,제한없음,
3647,한국전기차충전서비스,다산제2공영주차장,1,DC차데모+DC콤보,경기도,NaN,경기도 남양주시,제한없음,
3648,한국전기차충전서비스,다산제2공영주차장,2,AC완속,경기도,NaN,경기도 남양주시,제한없음,
3771,한국전기차충전서비스,문원동 공영주차장,1,DC차데모+DC콤보,경기도,NaN,경기도 과천시,제한없음,
3818,한국전기차충전서비스,별내제1공영주차장,1,DC차데모+DC콤보,경기도,NaN,경기도 남양주시,제한없음,


In [18]:
# 결측치가 있는 행의 부실한 주소 변경을위한 인덱스 값 확인
charge_gyeonggi[charge_gyeonggi['주소2'] == ''].index

Int64Index([877, 891, 892, 893, 894, 3553, 3647, 3648, 3771, 3818, 3819], dtype='int64')

In [19]:
# 인덱스 값 리스트화
gyeonggi_list = [877, 891, 892, 893, 894, 3553, 3647, 3648, 3771, 3818, 3819]

In [20]:
# 탐색을 통한 주소 확인 및 리스트화 
gyeonggi_addr = ['경기도 화성시 CU동탄금곡로점',
                 '경기도 고양시 N2WASH 일산전기차충전소',
                 '경기도 고양시 N2WASH 일산전기차충전소',
                 '경기도 고양시 N2WASH 일산전기차충전소',
                 '경기도 고양시 N2WASH 일산전기차충전소',
                 '경기도 남양주시 금곡제1공영주차장',
                 '경기도 남양주시 다산제2공영주차장',
                 '경기도 남양주시 다산제2공영주차장',
                 '경기도 과천시 문원동 공영주차장',
                 '경기도 남양주시 별내제1공영주차장',
                 '경기도 남양주시 별내제1공영주차장']

In [21]:
# 부실한 주소 처리
charge_gyeonggi.iloc[gyeonggi_list, -3] = gyeonggi_addr

In [22]:
# 주소 처리 확인
charge_gyeonggi[charge_gyeonggi['주소2'] == '']

,운영기관,충전소,충전기ID,충전기타입,지역,시군구,주소,이용자 제한,주소2
877,제주전기자동차서비스,CU동탄금곡로점,1,DC콤보,경기도,NaN,경기도 화성시 CU동탄금곡로점,제한없음,
891,제주전기자동차서비스,N2WASH 일산전기차충전소,1,DC차데모+DC콤보,경기도,NaN,경기도 고양시 N2WASH 일산전기차충전소,제한없음,
892,제주전기자동차서비스,N2WASH 일산전기차충전소,2,DC차데모+DC콤보,경기도,NaN,경기도 고양시 N2WASH 일산전기차충전소,제한없음,
893,제주전기자동차서비스,N2WASH 일산전기차충전소,3,AC완속,경기도,NaN,경기도 고양시 N2WASH 일산전기차충전소,제한없음,
894,제주전기자동차서비스,N2WASH 일산전기차충전소,4,AC완속,경기도,NaN,경기도 고양시 N2WASH 일산전기차충전소,제한없음,
3553,한국전기차충전서비스,금곡제1공영주차장,1,DC차데모+DC콤보,경기도,NaN,경기도 남양주시 금곡제1공영주차장,제한없음,
3647,한국전기차충전서비스,다산제2공영주차장,1,DC차데모+DC콤보,경기도,NaN,경기도 남양주시 다산제2공영주차장,제한없음,
3648,한국전기차충전서비스,다산제2공영주차장,2,AC완속,경기도,NaN,경기도 남양주시 다산제2공영주차장,제한없음,
3771,한국전기차충전서비스,문원동 공영주차장,1,DC차데모+DC콤보,경기도,NaN,경기도 과천시 문원동 공영주차장,제한없음,
3818,한국전기차충전서비스,별내제1공영주차장,1,DC차데모+DC콤보,경기도,NaN,경기도 남양주시 별내제1공영주차장,제한없음,


In [23]:
# 시군구의 결측치 처리
charge_gyeonggi['시군구'] = charge_gyeonggi['주소'].str.split(' ', expand=True)[1]

In [24]:
# 최종 확인
charge_gyeonggi[charge_gyeonggi['주소2'] == '']

,운영기관,충전소,충전기ID,충전기타입,지역,시군구,주소,이용자 제한,주소2
877,제주전기자동차서비스,CU동탄금곡로점,1,DC콤보,경기도,화성시,경기도 화성시 CU동탄금곡로점,제한없음,
891,제주전기자동차서비스,N2WASH 일산전기차충전소,1,DC차데모+DC콤보,경기도,고양시,경기도 고양시 N2WASH 일산전기차충전소,제한없음,
892,제주전기자동차서비스,N2WASH 일산전기차충전소,2,DC차데모+DC콤보,경기도,고양시,경기도 고양시 N2WASH 일산전기차충전소,제한없음,
893,제주전기자동차서비스,N2WASH 일산전기차충전소,3,AC완속,경기도,고양시,경기도 고양시 N2WASH 일산전기차충전소,제한없음,
894,제주전기자동차서비스,N2WASH 일산전기차충전소,4,AC완속,경기도,고양시,경기도 고양시 N2WASH 일산전기차충전소,제한없음,
3553,한국전기차충전서비스,금곡제1공영주차장,1,DC차데모+DC콤보,경기도,남양주시,경기도 남양주시 금곡제1공영주차장,제한없음,
3647,한국전기차충전서비스,다산제2공영주차장,1,DC차데모+DC콤보,경기도,남양주시,경기도 남양주시 다산제2공영주차장,제한없음,
3648,한국전기차충전서비스,다산제2공영주차장,2,AC완속,경기도,남양주시,경기도 남양주시 다산제2공영주차장,제한없음,
3771,한국전기차충전서비스,문원동 공영주차장,1,DC차데모+DC콤보,경기도,과천시,경기도 과천시 문원동 공영주차장,제한없음,
3818,한국전기차충전서비스,별내제1공영주차장,1,DC차데모+DC콤보,경기도,남양주시,경기도 남양주시 별내제1공영주차장,제한없음,


In [25]:
# 확인차 사용한 주소2 컬럼 제거
charge_gyeonggi = charge_gyeonggi.drop(columns='주소2')

In [26]:
# 이용자 제한이 있는 충전소와 없는 충전소 구분
charge_gyeonggi_all = charge_gyeonggi[charge_gyeonggi['이용자 제한'] == '제한없음'].reset_index(drop=True)
charge_gyeonggi_else = charge_gyeonggi[charge_gyeonggi['이용자 제한'] != '제한없음'].reset_index(drop=True)

In [27]:
charge_gyeonggi_all

,운영기관,충전소,충전기ID,충전기타입,지역,시군구,주소,이용자 제한
0,환경부(한국자동차환경협회),KH양주휴게소,1,DC차데모+AC3상+DC콤보,경기도,양주시,경기도 양주시 장흥면 서울외곽순환고속도로 58,제한없음
1,환경부(한국자동차환경협회),S-oil 셀프 신현주유소(영광셀프주유소),1,DC차데모+AC3상+DC콤보,경기도,시흥시,경기도 시흥시 시흥대로869,제한없음
2,환경부(한국자동차환경협회),VIC마켓 킨텍스점,1,DC차데모+AC3상+DC콤보,경기도,고양시,경기도 고양시 일산서구 주엽로 79,제한없음
3,환경부(한국자동차환경협회),VIC마켓 킨텍스점,2,DC차데모+AC3상+DC콤보,경기도,고양시,경기도 고양시 일산서구 주엽로 79,제한없음
4,환경부(한국자동차환경협회),㈜디에스 화랑포 셀프주유소,91,DC콤보,경기도,가평군,경기도 가평군 청평면 경춘로 113,제한없음
...,...,...,...,...,...,...,...,...
11341,캐스트프로,캐스트프로테스,2,DC차데모+AC3상,경기도,성남시,경기도 성남시 중원구 둔촌대로457번길 2713F(311-3 성남우림라이온스밸리),제한없음
11342,한화솔루션,파주유앤비빌딩,1,AC완속,경기도,파주시,경기도 파주시 금릉이길 1유앤비빌딩(793-4 유앤비빌딩),제한없음
11343,한화솔루션,파주유앤비빌딩,2,AC완속,경기도,파주시,경기도 파주시 금릉이길 1유앤비빌딩(793-4 유앤비빌딩),제한없음
11344,한화솔루션,파주유앤비빌딩,3,DC콤보,경기도,파주시,경기도 파주시 금릉이길 1유앤비빌딩(793-4 유앤비빌딩),제한없음


In [28]:
charge_gyeonggi_else

,운영기관,충전소,충전기ID,충전기타입,지역,시군구,주소,이용자 제한
0,파킹클라우드,고양 삼송역현대헤리엇아파트,1,AC완속,경기도,고양시,경기도 고양시 덕양구 삼송로 222(현대 헤리엇),거주자외 출입제한
1,파킹클라우드,고양 삼송역현대헤리엇아파트,2,AC완속,경기도,고양시,경기도 고양시 덕양구 삼송로 222(현대 헤리엇),거주자외 출입제한
2,파킹클라우드,고양 삼송역현대헤리엇아파트,3,AC완속,경기도,고양시,경기도 고양시 덕양구 삼송로 222(현대 헤리엇),거주자외 출입제한
3,파킹클라우드,고양 삼송역현대헤리엇아파트,4,AC완속,경기도,고양시,경기도 고양시 덕양구 삼송로 222(현대 헤리엇),거주자외 출입제한
4,파킹클라우드,고양 삼송역현대헤리엇아파트,5,AC완속,경기도,고양시,경기도 고양시 덕양구 삼송로 222(현대 헤리엇),거주자외 출입제한
...,...,...,...,...,...,...,...,...
18746,중앙제어,롯데인재개발원 오산캠퍼스,4,AC완속,경기도,오산시,경기도 오산시 경기동로186번길 8롯데인재개발원(부산동 88),교육생 및 관련자외 출입제한
18747,중앙제어,롯데인재개발원 오산캠퍼스,5,AC완속,경기도,오산시,경기도 오산시 경기동로186번길 8롯데인재개발원(부산동 88),교육생 및 관련자외 출입제한
18748,중앙제어,롯데인재개발원 오산캠퍼스,6,AC완속,경기도,오산시,경기도 오산시 경기동로186번길 8롯데인재개발원(부산동 88),교육생 및 관련자외 출입제한
18749,중앙제어,롯데인재개발원 오산캠퍼스,7,AC완속,경기도,오산시,경기도 오산시 경기동로186번길 8롯데인재개발원(부산동 88),교육생 및 관련자외 출입제한


In [29]:
# 정제된 데이터 내보내기
charge_gyeonggi_all.to_csv('./data/2.충전소/충전소_경기도_제한없음.csv')
charge_gyeonggi_else.to_csv('./data/2.충전소/충전소_경기도_제한있음.csv')